In [2]:
import os
from random import randint

import cv2
import numpy as np
from falatra.markers import MarkerDetection
from falatra.keypoints import Frame, FrameMatcher

from matplotlib import pyplot as plt

In [3]:
def listFullPaths(folder):
    
    paths = []
    for filename in sorted(os.listdir(folder)):
        path = os.path.join(folder, filename)
        paths.append(path)
        
    return paths

training = {}
training['side'] = listFullPaths('./data/training/segmented1/left')
training['front'] = listFullPaths('./data/training/segmented1/centre')

labels = {}
labels['side'] = listFullPaths('./data/training/segmented1/left_labels')
labels['front'] = listFullPaths('./data/training/segmented1/centre_labels')

xarr = []
yarr = []
n_points = 0
for image_index in range(len(training['side']) - 1):

    image_front = cv2.imread(training['front'][0])
    label_front = MarkerDetection()
    label_front.load(labels['front'][0])

    frame_front = Frame(image_front)
    frame_front.detect(detectFace=True)
    
    n_points += len(frame_front.kps)
    for i, keypoint in enumerate(frame_front.kps):
        x, y = keypoint.pt
        xarr.append(x)
        yarr.append(y)

In [23]:
%matplotlib qt

xarr = np.array(xarr)
yarr = np.array(yarr)

fig, ax = plt.subplots(1, 1, figsize=(8,6))
plt.title('Keypoints accumulator')
h = plt.hist2d(xarr, yarr, bins=(50, 50), cmap=plt.cm.jet, density=True)
cbar = fig.colorbar(h[3], ax=ax)
cbar.set_label('Density')
cbar.ax.tick_params(labelsize='small')
ax.invert_yaxis()
    
kptsImg = frame_front.getKeypointsVisual()
plt.figure()
plt.imshow(kptsImg[...,[2,1,0]])

plt.show()

In [24]:
xarr = []
yarr = []
n_points = 0
for image_index in range(len(training['side']) - 1):

    image_side = cv2.imread(training['side'][0])
    label_side = MarkerDetection()
    label_side.load(labels['side'][0])

    frame_side = Frame(image_side)
    frame_side.detect(detectFace=True)
    
    n_points += len(frame_side.kps)
    for i, keypoint in enumerate(frame_side.kps):
        x, y = keypoint.pt
        xarr.append(x)
        yarr.append(y)

In [28]:
xarr = np.array(xarr)
yarr = np.array(yarr)

fig, ax = plt.subplots(1, 1, figsize=(8,6))
plt.title('Keypoints accumulator')
h = plt.hist2d(xarr, yarr, bins=(50, 50), cmap=plt.cm.jet, density=True)
cbar = fig.colorbar(h[3], ax=ax)
cbar.set_label('Density')
cbar.ax.tick_params(labelsize='small')
ax.invert_yaxis()

kptsImg = frame_side.getKeypointsVisual()
plt.figure()
plt.imshow(kptsImg[...,[2,1,0]])
plt.show()

In [49]:
from falatra.utils import find_bbox_centre, distance_from_line
from falatra.model.stereo import StereoCalibration

def lowe_ratio_filter(knnmatches, loweratio=0.7):
    
    good = []
    for knnmatch in knnmatches:
        m, n = knnmatch[:2]  # get the 1st and 2nd closest
        if m.distance < loweratio * n.distance:
            good.append([m])
            
    return good

def display_matches(matches, queryimg, querykps, trainimg, trainkps):
    vis = cv2.drawMatchesKnn(queryimg, querykps, 
                             trainimg, trainkps, 
                             matches,
                             None,
                             flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure()
    plt.imshow(vis[...,[2,1,0]])
    plt.show()

calibration = StereoCalibration()
calibration.load('./data/calibration/calibration1')
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

xarr_centre = []
yarr_centre = []
xarr_left = []
yarr_left = []

for image_index in range(len(training['side']) - 1):
    image_front = cv2.imread(training['front'][0])
    label_front = MarkerDetection()
    label_front.load(labels['front'][0])

    frame_front = Frame(image_front)
    frame_front.detect(detectFace=True)
    
    image_side = cv2.imread(training['side'][0])
    label_side = MarkerDetection()
    label_side.load(labels['side'][0])

    frame_side = Frame(image_side)
    frame_side.detect(detectFace=True)
    

    # ---------- 1st step: FIND CORRESPONDENCES BETWEEN STEREO IMAGES ---------- 

    stereomatches = matcher.knnMatch(frame_side.des, frame_front.des, 2)

    # Lowe ratio filtering
    loweratio = 0.9
    stereomatches = lowe_ratio_filter(stereomatches, loweratio)

    # Epipolar constraint
    epipolar_threshold = 20.0
    good = []
    for match in stereomatches:

        pt_src = frame_side.kps[match[0].queryIdx].pt
        pt_dst = frame_front.kps[match[0].trainIdx].pt

        # undistort these points
        pt_src = calibration.undistort_points([pt_src], view='right').squeeze()
        pt_dst = calibration.undistort_points([pt_dst], view='left').squeeze()

        pt_dst = pt_dst[np.newaxis, ...]  # junk to make this work if my code wasn't so bad
        line = calibration.compute_correspond_epilines(pt_dst, view='right').squeeze()

        d = distance_from_line(pt_src, *line)
        if d < epipolar_threshold:
            good.append(match)
    stereomatches = good

    for match in stereomatches:
        x, y = frame_front.kps[match[0].trainIdx].pt
        xarr_centre.append(x)
        yarr_centre.append(y)

        x, y = frame_side.kps[match[0].queryIdx].pt
        xarr_left.append(x)
        yarr_left.append(y)

        


In [50]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,6))
h = ax1.hist2d(xarr_centre, yarr_centre, bins=(50, 50), cmap=plt.cm.jet, density=True)
ax1.set_title('Keypoint accumulator centre')
cbar = fig.colorbar(h[3], ax=ax1)
cbar.set_label('Density')
cbar.ax.tick_params(labelsize='small')
ax1.invert_yaxis()

h = ax2.hist2d(xarr_left, yarr_left, bins=(50, 50), cmap=plt.cm.jet, density=True)
ax2.set_title('Keypoint accumulator left')
cbar = fig.colorbar(h[3], ax=ax2)
cbar.set_label('Density')
cbar.ax.tick_params(labelsize='small')
ax2.invert_yaxis()

display_matches(stereomatches, frame_side.image, frame_side.kps, frame_front.image, frame_front.kps)